In [ ]:
#!/usr/bin/env python3
"""
Main Behavior Tree Node untuk Robot Control
Implementasi menggunakan py_trees dan ROS2
"""

import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist, PoseStamped
from sensor_msgs.msg import LaserScan, BatteryState
from nav_msgs.msg import Odometry
import py_trees
from py_trees.behaviour import Behaviour
from py_trees.composites import Selector, Sequence
from py_trees.decorators import FailureIsSuccess, SuccessIsFailure
import math
import numpy as np


class BatteryCheckCondition(Behaviour):
    """Check apakah battery level rendah"""

    def __init__(self, name, battery_threshold=20.0):
        super().__init__(name)
        self.battery_threshold = battery_threshold
        self.battery_level = 100.0

    def update(self):
        if self.battery_level < self.battery_threshold:
            self.feedback_message = f"Battery low: {self.battery_level}%"
            return py_trees.common.Status.SUCCESS
        else:
            return py_trees.common.Status.FAILURE


class ObstacleDetectionCondition(Behaviour):
    """Deteksi obstacle di depan robot"""

    def __init__(self, name, safe_distance=0.5):
        super().__init__(name)
        self.safe_distance = safe_distance
        self.min_distance = float('inf')

    def update(self):
        if self.min_distance < self.safe_distance:
            self.feedback_message = f"Obstacle detected at {self.min_distance:.2f}m"
            return py_trees.common.Status.SUCCESS
        return py_trees.common.Status.FAILURE


class TargetDetectedCondition(Behaviour):
    """Check apakah target terdeteksi"""

    def __init__(self, name):
        super().__init__(name)
        self.target_detected = False
        self.target_position = None

    def update(self):
        if self.target_detected:
            self.feedback_message = "Target detected"
            return py_trees.common.Status.SUCCESS
        return py_trees.common.Status.FAILURE


class GoToChargingAction(Behaviour):
    """Action untuk pergi ke charging station"""

    def __init__(self, name, node, charging_position):
        super().__init__(name)
        self.node = node
        self.charging_position = charging_position
        self.reached = False

    def initialise(self):
        self.reached = False
        self.feedback_message = "Moving to charging station"

    def update(self):
        # Cek jarak ke charging station
        current_pos = self.node.current_position
        if current_pos is None:
            return py_trees.common.Status.RUNNING

        distance = math.sqrt(
            (self.charging_position[0] - current_pos[0])**2 +
            (self.charging_position[1] - current_pos[1])**2
        )

        if distance < 0.3:
            self.feedback_message = "Reached charging station"
            return py_trees.common.Status.SUCCESS

        # Navigate ke charging station
        self.node.navigate_to_goal(self.charging_position)
        return py_trees.common.Status.RUNNING

    def terminate(self, new_status):
        self.node.stop_robot()


class AvoidObstacleAction(Behaviour):
    """Action untuk menghindari obstacle"""

    def __init__(self, name, node):
        super().__init__(name)
        self.node = node
        self.avoiding = False

    def initialise(self):
        self.avoiding = True
        self.feedback_message = "Avoiding obstacle"

    def update(self):
        # Rotate robot untuk menghindari obstacle
        cmd = Twist()
        cmd.linear.x = -0.2
        cmd.angular.z = 0.5
        self.node.cmd_vel_pub.publish(cmd)

        # Setelah beberapa saat, anggap berhasil
        return py_trees.common.Status.RUNNING

    def terminate(self, new_status):
        self.node.stop_robot()


class ApproachTargetAction(Behaviour):
    """Action untuk mendekati target yang terdeteksi"""

    def __init__(self, name, node):
        super().__init__(name)
        self.node = node

    def initialise(self):
        self.feedback_message = "Approaching target"

    def update(self):
        if not hasattr(self.node, 'target_position') or self.node.target_position is None:
            return py_trees.common.Status.FAILURE

        # Navigate ke target
        self.node.navigate_to_goal(self.node.target_position)
        return py_trees.common.Status.RUNNING


class PatrolAction(Behaviour):
    """Action untuk patrol waypoints"""

    def __init__(self, name, node, waypoints):
        super().__init__(name)
        self.node = node
        self.waypoints = waypoints
        self.current_waypoint_idx = 0

    def initialise(self):
        self.feedback_message = f"Patrolling waypoint {self.current_waypoint_idx + 1}"

    def update(self):
        current_pos = self.node.current_position
        if current_pos is None:
            return py_trees.common.Status.RUNNING

        target = self.waypoints[self.current_waypoint_idx]
        distance = math.sqrt(
            (target[0] - current_pos[0])**2 +
            (target[1] - current_pos[1])**2
        )

        if distance < 0.5:
            # Reached waypoint, move to next
            self.current_waypoint_idx = (self.current_waypoint_idx + 1) % len(self.waypoints)
            self.feedback_message = f"Reached waypoint, moving to {self.current_waypoint_idx + 1}"

        # Navigate to current waypoint
        self.node.navigate_to_goal(target)
        return py_trees.common.Status.RUNNING

    def terminate(self, new_status):
        self.node.stop_robot()


class BehaviorTreeNode(Node):
    """ROS2 Node untuk menjalankan Behavior Tree"""

    def __init__(self):
        super().__init__('behavior_tree_node')

        # Publishers
        self.cmd_vel_pub = self.create_publisher(Twist, '/cmd_vel', 10)

        # Subscribers
        self.create_subscription(LaserScan, '/scan', self.laser_callback, 10)
        self.create_subscription(Odometry, '/odom', self.odom_callback, 10)
        self.create_subscription(BatteryState, '/battery_state', self.battery_callback, 10)

        # Robot state
        self.current_position = None
        self.current_orientation = None
        self.battery_level = 100.0
        self.min_obstacle_distance = float('inf')
        self.target_position = None

        # Parameters
        self.declare_parameter('tick_rate', 10.0)
        self.declare_parameter('battery_threshold', 20.0)
        self.declare_parameter('safe_distance', 0.5)

        # Waypoints untuk patrol
        self.patrol_waypoints = [
            [2.0, 2.0],
            [2.0, -2.0],
            [-2.0, -2.0],
            [-2.0, 2.0]
        ]

        self.charging_position = [0.0, 0.0]

        # Build behavior tree
        self.tree = self.create_behavior_tree()

        # Timer untuk tick behavior tree
        tick_rate = self.get_parameter('tick_rate').value
        self.timer = self.create_timer(1.0 / tick_rate, self.tick_tree)

        self.get_logger().info('Behavior Tree Node initialized')

    def create_behavior_tree(self):
        """Buat struktur behavior tree"""

        battery_threshold = self.get_parameter('battery_threshold').value
        safe_distance = self.get_parameter('safe_distance').value

        # Conditions
        battery_check = BatteryCheckCondition("Battery Low?", battery_threshold)
        obstacle_check = ObstacleDetectionCondition("Obstacle Ahead?", safe_distance)
        target_check = TargetDetectedCondition("Target Detected?")

        # Actions
        go_charging = GoToChargingAction("Go to Charging", self, self.charging_position)
        avoid_obstacle = AvoidObstacleAction("Avoid Obstacle", self)
        approach_target = ApproachTargetAction("Approach Target", self)
        patrol = PatrolAction("Patrol", self, self.patrol_waypoints)

        # Update references untuk conditions
        battery_check.blackboard = self
        obstacle_check.blackboard = self
        target_check.blackboard = self

        # Build tree structure
        # Root: Selector (priority behaviors)
        root = Selector(name="Root", memory=False)

        # Priority 1: Battery management
        battery_sequence = Sequence(name="Battery Management", memory=True)
        battery_sequence.add_children([battery_check, go_charging])

        # Priority 2: Obstacle avoidance
        obstacle_sequence = Sequence(name="Obstacle Avoidance", memory=True)
        obstacle_sequence.add_children([obstacle_check, avoid_obstacle])

        # Priority 3: Target approach
        target_sequence = Sequence(name="Target Approach", memory=True)
        target_sequence.add_children([target_check, approach_target])

        # Add all behaviors to root
        root.add_children([
            battery_sequence,
            obstacle_sequence,
            target_sequence,
            patrol
        ])

        return py_trees.trees.BehaviourTree(root)

    def tick_tree(self):
        """Tick behavior tree setiap cycle"""

        # Update blackboard data untuk conditions
        for node in self.tree.root.iterate():
            if isinstance(node, BatteryCheckCondition):
                node.battery_level = self.battery_level
            elif isinstance(node, ObstacleDetectionCondition):
                node.min_distance = self.min_obstacle_distance
            elif isinstance(node, TargetDetectedCondition):
                node.target_detected = (self.target_position is not None)

        # Tick tree
        self.tree.tick()

        # Log status
        status = self.tree.root.status
        self.get_logger().info(f'Tree status: {status}', throttle_duration_sec=2.0)

    def laser_callback(self, msg):
        """Callback untuk laser scan data"""
        valid_ranges = [r for r in msg.ranges if msg.range_min < r < msg.range_max]
        if valid_ranges:
            self.min_obstacle_distance = min(valid_ranges)
        else:
            self.min_obstacle_distance = float('inf')

    def odom_callback(self, msg):
        """Callback untuk odometry data"""
        self.current_position = [
            msg.pose.pose.position.x,
            msg.pose.pose.position.y
        ]
        self.current_orientation = msg.pose.pose.orientation

    def battery_callback(self, msg):
        """Callback untuk battery state"""
        self.battery_level = msg.percentage * 100.0

    def navigate_to_goal(self, goal_position):
        """Navigate robot ke goal position"""
        if self.current_position is None:
            return

        # Hitung angle ke goal
        dx = goal_position[0] - self.current_position[0]
        dy = goal_position[1] - self.current_position[1]
        target_angle = math.atan2(dy, dx)

        # Get current yaw dari quaternion
        current_yaw = self.get_yaw_from_quaternion(self.current_orientation)

        # Hitung error angle
        angle_error = target_angle - current_yaw
        angle_error = math.atan2(math.sin(angle_error), math.cos(angle_error))

        # Simple proportional controller
        cmd = Twist()
        cmd.linear.x = min(0.3, math.sqrt(dx**2 + dy**2))
        cmd.angular.z = 2.0 * angle_error

        self.cmd_vel_pub.publish(cmd)

    def stop_robot(self):
        """Stop robot movement"""
        cmd = Twist()
        self.cmd_vel_pub.publish(cmd)

    def get_yaw_from_quaternion(self, q):
        """Convert quaternion to yaw angle"""
        if q is None:
            return 0.0
        siny_cosp = 2 * (q.w * q.z + q.x * q.y)
        cosy_cosp = 1 - 2 * (q.y * q.y + q.z * q.z)
        return math.atan2(siny_cosp, cosy_cosp)


def main(args=None):
    rclpy.init(args=args)
    node = BehaviorTreeNode()

    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()